In [1]:
# import libraries

from pyspark.sql import SparkSession
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS, ALSModel
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
import datetime


In [2]:
# make a SparkSession object

spark = (SparkSession
         .builder
         .appName("MoviesALS")
         .config("spark.driver.host", "localhost")
         .getOrCreate())

In [3]:
# import ratings json file into spark dataframe

movie_ratings = spark.read.json('data/ratings.json')

In [4]:
# check schema
movie_ratings.printSchema()

root
 |-- movie_id: long (nullable = true)
 |-- rating: long (nullable = true)
 |-- timestamp: double (nullable = true)
 |-- user_id: long (nullable = true)



In [5]:
# cast to Pandas dataframe to turn timestamp data to datetime and check nulls. 

movies_df = movie_ratings.select('*').toPandas()
movies_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 719949 entries, 0 to 719948
Data columns (total 4 columns):
movie_id     719949 non-null int64
rating       719949 non-null int64
timestamp    719949 non-null float64
user_id      719949 non-null int64
dtypes: float64(1), int64(3)
memory usage: 22.0 MB


In [7]:
# attempt to change timestamp object to years, all years are 2000

date = pd.to_datetime(movies_df['timestamp'], unit='s').dt.year
date.value_counts()

2000    719949
Name: timestamp, dtype: int64

In [8]:
# Decide to drop timestamp for now because only year 2000

movie_ratings = movie_ratings.drop('timestamp')

In [9]:
# Split data into training and test set

(training, test) = movie_ratings.randomSplit([.8, .2])

In [11]:
# Create ALS instance and fit model

als = ALS(maxIter=10,
          rank=10,
          userCol='user_id',
          itemCol='movie_id',
          ratingCol='rating')

model = als.fit(training)

In [13]:
# Generate Predictions

predictions = model.transform(test)
predictions.persist()

DataFrame[movie_id: bigint, rating: bigint, user_id: bigint, prediction: float]

In [18]:
# Convert to pandas dataframe, fill prediction nulls, and convert back to spark dataframe

pred_df = predictions.select('*').toPandas()

def user_average(user, df):
    """Return average score for user"""
    user_df = df[df['user_id'] == user]
    average = user_df['prediction'].mean()
    if np.isnan(average):
        return 3
    else:
        return average
    
def compute_user_average_if_null(row):
    """Check if value is null, if so, replace with user average"""
    if np.isnan(row['prediction']):
        return user_average(row['user_id'], pred_df)
    else:
        return row['prediction']
    
for i, row in pred_df[pred_df['prediction'].isna()].iterrows():
    pred_df.loc[i, 'prediction'] = compute_user_average_if_null(row)
    
print(pred_df['prediction'].isna().any())
    
predictions = spark.createDataFrame(pred_df)

False


In [22]:
# Evaluate model 
evaluator = RegressionEvaluator(metricName='rmse', labelCol='rating',
                               predictionCol='prediction')

rmse = evaluator.evaluate(predictions)
print(rmse)

0.8770237505561516


In [ ]:
# Create a parameter grid, cross validate for best model with different hyperperameters
params_score = {}

params = (ParamGridBuilder()
          .addGrid(als.regParam, [1, 0.01, 0.001, 0.1])
          .addGrid(als.maxIter, [5, 10, 20])
          .addGrid(als.rank, [4, 10, 50])).build()

cv = CrossValidator(estimator=als, estimatorParamMaps=params, evaluator=evaluator, parallelism=4)

best_model = cv.fit(movie_ratings)
als_model = best_model.bestModel

In [ ]:
# save model
als_model.save('als_model')

In [ ]:
# load requests json file into a spark dataframe
    
requests = spark.read.json("data/requests.json") 

# predict requests with best model
requests_predictions = als_model.transform(requests)

# export request predictions dataframe as json file.
